In [2]:
import numpy as np
from random import random
from scipy.optimize import minimize

from qiskit import *
from qiskit.circuit.library import U2Gate
from qiskit.quantum_info import Pauli, SparsePauliOp, Operator
from qiskit.visualization import plot_bloch_multivector, plot_histogram
from qiskit_aer import Aer
from qiskit_algorithms import NumPyEigensolver



In [3]:

def hamiltonian_operator(a, b, c, d):

    return SparsePauliOp.from_list([("I", a), ("Z", b) , ("X", c), ("Y", d)])

scale = 10
a, b, c, d = (scale*random(), scale*random(), scale*random(), scale*random() )
H = hamiltonian_operator(a, b,c, d)

print(H)
   

SparsePauliOp(['I', 'Z', 'X', 'Y'],
              coeffs=[5.69669392+0.j, 0.66144616+0.j, 5.87595241+0.j, 7.23195887+0.j])


In [4]:
# This is for the comparison with the classical algorithm
solver = NumPyEigensolver()
result = solver.compute_eigenvalues(Operator(H))
reference_energy = min(np.real(result.eigenvalues))
print(reference_energy)

-3.644909627835048


In [5]:
# this fucntion gives us all possible ansatz states

def ansatz_state_prep(circuit:QuantumCircuit, parameters):
    q = circuit.qregs[0]
    circuit.rx(parameters[0], q[0])
    circuit.ry(parameters[1], q[0])

    return circuit

In [6]:
# since we need to perform measurements in Z, X, Y with Zbasis already being available 
# we then define the following gates for measurements in X and Y basis

# measurement in X basis

def x_measurement(qc, qbit, cbit):
    qc.h(qbit)
    qc.measure(qbit, cbit)
    return None

# measurement in Y basis

def y_measurement(qc, qbit, cbit):
    minus_pi = -np.pi
    qc.p(minus_pi/2, qbit)
    qc.h(qbit)
    qc.p(minus_pi/2, qbit)
    qc.measure(qbit, cbit)
    return None


In [7]:
# Vqe circuit

def vqe_circuit(parameters, measure: str):
    circuit = QuantumCircuit(1,1)
    circuit = ansatz_state_prep(circuit, parameters)
    if measure == 'Z':
        circuit.measure(0, 0)
    elif measure == 'X':
        x_measurement(circuit, 0, 0)
    elif measure == 'Y':
        y_measurement(circuit, 0, 0)
    else:
        raise ValueError('Not valid input for measurement: input should be "X" or "Y" or "Z"')
    return circuit

In [8]:
# finding the expectation values of a Pauli operator

def quantum_module(parameters, measure):
    if measure == 'I':
        return 1
    else:
        circuit = vqe_circuit(parameters, measure)
    
    shots = 8192 # 2**13
    sim = Aer.get_backend('aer_simulator')
    result = sim.run(circuit, shots = shots).result()

    counts = result.get_counts()
    # plot_histogram(counts)

    expectation_value = 0
    for measure_result in counts.keys():
        sign = +1
        if measure_result == '1':
            sign = -1
        expectation_value += sign*counts[measure_result] /shots
    return expectation_value

In [9]:
def vqe(parameters):
    modules = []
    classical_adder = 0 
    for x in H:
        modules.append(np.real(x.coeffs[0]) * quantum_module(parameters, str(x.paulis[0])))

    for mod in modules:
        classical_adder += mod
    return classical_adder

In [10]:
parameters_array = np.array([np.pi, np.pi])
tol = 1e-3

vqe_result = minimize(vqe, parameters_array, method='Powell', tol = tol)
print(f'The exact ground state energy is: {reference_energy}')
print('The estimated ground state energy from VQE algorithm is: {}'.format(vqe_result.fun))

The exact ground state energy is: -3.644909627835048
The estimated ground state energy from VQE algorithm is: -3.5480107356592465
